In [23]:
from langgraph.graph import StateGraph,START, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict): 
    text: str

def node_a(state:State):
    print ("Node A")
    return Command(
        goto="node_b",
        update={
            "text": state["text"] + "a"
        }
    )
def node_b(state:State):
    print ("Node B")

    human_response = interrupt("Do you want to go to C or D? Type C/D")

    print ("Human Review Values: ", human_response)

    if(human_response == "C"):
        return Command(
            goto="node_c",
            update={
                "text": state["text"] + "b - C"
            }
        )
    elif(human_response == "D"):
        return Command(
            goto="node_d",
            update={
                "text": state["text"] + "b - D"
            }
        )

def node_c(state:State):
    print ("Node C")
    return Command(
        goto=END,
        update={
            "text": state["text"] + "c"
        }
    )
def node_d(state:State):
    print ("Node D")
    return Command(
        goto=END,
        update={
            "text": state["text"] + "d "
        }
    )

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a")
app = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

initialState = {
    "text": "diego"
}

first_result = app.invoke(initialState, config, stream_mode="updates")
first_result

Node A
Node B


[{'node_a': {'text': 'diegoa'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', resumable=True, ns=['node_b:25c41a78-9617-c16f-f1d3-bd0281bbd9f5']),)}]

In [24]:
print(app.get_state(config).next)

('node_b',)


In [25]:
second_result = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
second_result

Node B
Human Review Values:  C
Node C


[{'node_b': {'text': 'diegoab - C'}}, {'node_c': {'text': 'diegoab - Cc'}}]